In [ ]:
import numpy as np
from tqdm.notebook import tqdm
import requests

from energy import energy_gradient, energy_function_forward, gradient_magnitude_sobel_operator
from seams import seams_map_dp, seam_map_function_forward, carve_column_mask
from seam_carve import SeamCarve

from PIL import Image
import os

In [ ]:
# url = 'https://static.boredpanda.com/blog/wp-content/uploads/2016/03/hairy-fluffy-cat-sky-the-ragdoll-31.jpg'
# img = Image.open(requests.get(url, stream=True).raw)
# img.save(os.path.join('images/cat.jpg'))

In [ ]:
img_path = 'images/cat.jpg'

new_width = 650

## Energy gradient

In [ ]:
sc_dual_grad = SeamCarve(image_path=img_path,
                new_width=new_width,
                energy_function=energy_gradient,
                seam_map_function=seams_map_dp,
                carve_function=carve_column_mask)

sc_dual_grad.run()

In [ ]:
# sc_dual_grad.__dict__

In [ ]:
sc_dual_grad.save_result(name_suffix='dual_grad')

In [ ]:
np.sum(sc_dual_grad.time_by_step)

## Sobel

In [ ]:
sc_sobel = SeamCarve(image_path=img_path,
                new_width=new_width,
                energy_function=gradient_magnitude_sobel_operator,
                seam_map_function=seams_map_dp,
                carve_function=carve_column_mask)

sc_sobel.run()

In [ ]:
sc_sobel.save_result(name_suffix='sobel')

## Forward energy

In [ ]:
sc_forward = SeamCarve(image_path=img_path,
                new_width=new_width,
                energy_function=energy_function_forward,
                seam_map_function=seam_map_function_forward,
                carve_function=carve_column_mask)

sc_forward.run()

In [ ]:
sc_forward.save_result(name_suffix='forward')